## Batch deployment

Goals:
This is an example of turning a training notebook to a batch inferencing notebook.

* Turn te notebook for training a model into a notebook for applyin the model 
* Turn the notebook into a script
* Clean it and parameterize it


In [ ]:
import pandas as pd
import uuid
import pickle
import mlflow
import os

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import make_pipeline


In [ ]:
# parameterizing the input and output data 
year = 2021
month = 3

input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_{year:04d}-{month:02d}.parquet'
output_file = f'output/green_tripdata_{year:04d}-{month:02d}.parquet'

# other parameters
RUN_ID = os.getenv('RUN_ID', '74de625dfdf14d32a8a939330f51c35a')



In [ ]:
# functions
# function to generate uuid
def generate_uuids(n):
    ride_ids = []
    for i in range(n):
        ride_ids.append(str(uuid.uuid4()))
    return ride_ids
    

def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    # assigning unique ids to the rows in the df
    df['ride_id'] = generate_uuids(len(df))

    return df

def prepare_dictionaries(df: pd.DataFrame):
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts


In [ ]:
# functionize the model application steps

def load_model(run_id):
    logged_model = f'02-exp-tracking/mlruns/1/{RUN_ID}/artifacts/models_mlflow/model.xgb'  # the 1 is for the experiment id
    model = mlflow.pyfunc.load_model(logged_model)
    return model 


def apply_model(input_file, run_id, model_version, output_file):
    df = read_dataframe(input_file) # getting the data
    dicts = prepare_dictionaries(df)

    model = load_model(run_id)
    y_pred = model.predict(dicts)

    df_result = pd.DataFrame()
    df_result['ride_id'] = df['ride_id']
    df_result['lpep_pickup_datetime'] = df['lpep_pickup_datetime']
    df_result['PULocationID'] = df['PULocationID']
    df_result['DOLocationID'] = df['DOLocationID']
    df_result['actual_duration'] = df['duration']
    df_result['predicted_duration'] = y_pred
    df_result['diff'] = df_result['actual_duration'] - df_result['predicted_duration']
    df_result['model_version'] = RUN_ID
    # save new data 
    df_result.to_parquet(output_file, index=False)
    


In [ ]:
# create the output dir
!mkdir output

In [ ]:
# apply the model
apply_model(input_file= input_file,
            run_id= RUN_ID,
            output_file=output_file)


Converting the notebook to python script
* Run the command in terminal `jupyter nbconvert --to script batch_inferencing.ipynb` 